In [1]:
using Pkg
Pkg.activate("MyProject")

using JuMP
const jmp = JuMP

using GLPK

  Activating project at `~/GitHub/StabTheory/MyProject`


In [2]:
include("libs/cnc.jl"); include("libs/initializer.jl")

# $1$-qubit magic state:

Using stabilizer states as a quasiprobability model for simplicity.

In [3]:
n = 1; CNC1 = generate_all_cncs(n,[i for i in 1:n])
R1 = generate_cnc_vertices(1,CNC1)

4×8 Matrix{Rational{Int64}}:
  1   1   1   1   1   1   1  1
 -1  -1  -1   1   1  -1   1  1
  1  -1   1   1  -1  -1  -1  1
  1   1  -1  -1  -1  -1   1  1

In [4]:
T = [1,1/sqrt(2),0,1/sqrt(2)]

4-element Vector{Float64}:
 1.0
 0.7071067811865475
 0.0
 0.7071067811865475

In [5]:
InitT = Initializer(1,T,R1);

In [6]:
InitT.InitDist

Dict{MaximalCnc, Float64} with 3 entries:
  CNC:…                                                               => 0.5
  CNC:…                                                               => 0.146447
  CNC:…                                                               => 0.353553

# $2$-qubit Magic state:

In [7]:
n = 2; CNC2 = generate_all_cncs(n,[i for i in 1:n])
R2 = generate_cnc_vertices(n,CNC2)

16×432 Matrix{Rational{Int64}}:
  1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
  1   0   0   0   0   0   0   1   0      0   0   0   0   0   0   0   0   1
  0   0   0   0   0   0   0   0   1      0   0   0   0   1  -1   0   0   0
  0   0   1   1   0   0   0   1   0     -1   0  -1  -1   0   1   0   1   0
  0  -1  -1   1   0  -1  -1   0   0      0  -1   0   0   0   0   0  -1  -1
  0   0   0   0  -1   0   0   0  -1  …  -1   0   1  -1  -1   1   1   0  -1
  1   0   0   0   1   0   0   1   0     -1   0  -1   1   0   0  -1   0   0
 -1   0  -1   1   1   0   0   0   1      0   0   0   0   1   0   1  -1   0
 -1   1   0  -1  -1   0   1   0   0      0  -1   0   1   0   0   1   1  -1
 -1   0   1   0   0   1   0   0  -1      1   0  -1   0   1   1   0   0  -1
  0   0   1   0   0  -1   0   1   0  …  -1   0  -1   0   0   0   0   0   0
  0   0   0  -1   0   1   0   0  -1      0   0   0  -1   1   0   0   1   0
  0   0   0   1  -1   1   0   0   1      1   0  -1   0  -1   0   1  

In [8]:
TT = [1,1/sqrt(2),0,1/sqrt(2),1/sqrt(2),1/2,0,1/2,0,0,0,0,1/sqrt(2),1/2,0,1/2];

In [14]:
QQ = approximate_input(TT,3);

In [19]:
InitTT = Initializer(2,TT,R2);

In [20]:
InitTT.Feasible

true

In [21]:
InitTT.Mixture

"Convex"

In [22]:
InitTT.InitDist

Dict{MaximalCnc, Float64} with 15 entries:
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 6.93889e-17
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.0606602

# Maximally mixed state:

In [52]:
MIX = Vector{Float64}(vcat([1],[0 for i in 1:15]))

16-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [53]:
InitMIX = Initializer(2,MIX,R2);

In [55]:
InitMIX.Feasible

true

In [54]:
InitMIX.InitDist

Dict{MaximalCnc, Float64} with 13 entries:
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.142857
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                              => 0.0714286
  CNC:…                                                 

# 3-qubit magic state

In [25]:
n = 3; CNC3 = generate_all_cncs(n,[i for i in 1:n])
R3 = generate_cnc_vertices(n,CNC3)

64×71136 Matrix{Rational{Int64}}:
  1   1   1   1   1  1   1   1   1  1  …  1   1   1   1   1  1   1  1   1   1
 -1   0   0   0   0  0   0   0   0  1     0   0   0   0   0  0   1  0   0   0
  0   0   0   0   0  0   0   0   1  1     0   0   0   0   0  0  -1  0   0   0
  0   0   0   0   0  0   0   0   0  0     0   0   0   0   1  0   1  1   0   0
  0  -1   0   0   1  0   0   0   0  0     1   0  -1   0   0  1   0  0   0   0
  0   0  -1  -1   0  0   0   0   0  0  …  0   0   0  -1   0  0   0  0  -1   0
  0   0   0   0   0  0   0   0   0  0     0   1   0   0   0  0   0  0   0   1
  1   0   0  -1   0  0  -1   0   0  1     0   0   0   0   0  0   0  0   1   0
  0   0   0   0   0  0   0   0   0  0     0   0   0   0  -1  0   0  0   0   0
  0   0   0   0   0  0   1   0  -1  0     0   0  -1   0   0  1   0  0   0   0
  ⋮                  ⋮                 ⋱                  ⋮                 ⋮
  0   0   0   0   0  0   0   0   0  0  …  0   0  -1   0   1  0   1  0   0   0
  0   0   0  -1  -1  0   0   0

In [28]:
TTT = kron(kron(T,T),T)

64-element Vector{Float64}:
 1.0
 0.7071067811865475
 0.0
 0.7071067811865475
 0.7071067811865475
 0.4999999999999999
 0.0
 0.4999999999999999
 0.0
 0.0
 ⋮
 0.3535533905932737
 0.0
 0.0
 0.0
 0.0
 0.4999999999999999
 0.3535533905932737
 0.0
 0.3535533905932737

In [29]:
InitTTT = Initializer(3,TTT,R3);

In [38]:
InitTTT.Mixture

"Affine"

In [34]:
sum(values(InitTTT.InitDist))

0.9999999999968541

# Approximate magic state

In [52]:
QQQ = approximate_input(TTT,0)

64-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [53]:
InitQQQ = Initializer(3,QQQ,R3);

In [54]:
InitQQQ.Feasible

false

In [55]:
InitQQQ.Mixture

"Affine"

In [56]:
sum(values(InitQQQ.InitDist))

1.0000000000000722

In [57]:
length(InitQQQ.Supports)

60

# Partial magic states:

In [96]:
P = Vector{Float64}([1,1,0,0])
TPP = kron(kron(T,P),P)

64-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [81]:
InitTPP = Initializer(3,TPP,R3);

In [82]:
InitTPP.Feasible

true

In [83]:
InitTPP.Mixture

"Convex"

In [85]:
length(InitTPP.Supports)

11

In [86]:
InitTPP.InitDist

Dict{MaximalCnc, Float64} with 11 entries:
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.377961
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0244078
  CNC:…                                                              => 0.0488155
  CNC:…                                                              => 0.377961

In [87]:
TTP = kron(kron(T,T),P)

64-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 0.7071067811865475
 0.7071067811865475
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.4999999999999999
 0.4999999999999999
 0.0
 0.0

In [88]:
InitTTP = Initializer(3,TTP,R3);

In [89]:
InitTTP.Feasible

true

In [90]:
InitTTP.Mixture

"Convex"

In [91]:
InitTTP.InitDist

Dict{MaximalCnc, Float64} with 13 entries:
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.0606602
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.0428932
  CNC:…                                                            => 0.146447
  CNC:…                                                            => 0.0606602
 

# Stabilizer state:

In [62]:
bit2int = PauliString(3).bit_to_int
IXXX = [bit2int[[0,0,0,0,0,0]],bit2int[[1,0,0,0,0,0]],bit2int[[0,1,0,0,0,0]],bit2int[[0,0,1,0,0,0]],bit2int[[0,1,1,0,0,0]],bit2int[[1,0,1,0,0,0]],bit2int[[1,1,0,0,0,0]],bit2int[[1,1,1,0,0,0]]]

8-element Vector{Int64}:
  1
 17
  5
  2
  6
 18
 21
 22

In [74]:
XXX = zeros(Float64,64); XXX[IXXX] .= 1;

In [75]:
InitXXX = Initializer(3,XXX,R3);

In [76]:
InitXXX.Feasible

true

In [77]:
InitXXX.Mixture

"Convex"

In [78]:
InitXXX.InitDist

Dict{MaximalCnc, Float64} with 2 entries:
  CNC:…                                                                    => 0.5
  CNC:…                                                                    => 0.5